# Assignment on Facial Expression Recognition with CNN
The task in this assignment is to train a convolutional neural network that can recognize the facial expression from an image.

## Dataset
You will use the FER2013 dataset. The dataset is available in `torchvision`. For more details, see [here](https://pytorch.org/vision/main/generated/torchvision.datasets.FER2013.html#torchvision.datasets.FER2013).

## Architectures
In this assignment, you will experiment with the following architectures.
* Alexnet
* VGG11
* Resnet18
* Inception V3

Make necessary adjustments to the architecture to adapt it for you classification task. Do not use pre-trained weights.

## Training
* Train each model for several epochs
* Track the training loss and training accuracies as well as the testing loss and testing accuracies at each epoch (Note: This is only for simplicity. We typically do not use test set during the training).
*  Tune batch size and learning rates hyper-parameters. Experiment with at least 3 values of the hyper-parameter.

## Submission Guidelines
* Plot the training loss and test loss separately for each configuration (architecture, batch size, learning rate).
* Comment on the learning curves.
* Submit the following by 21 February, 2025
    * A PDF containing the learning curves and the discussions.
    * A `zip` file containing your codes.

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import Counter
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as T
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets

import spacy
from nltk.corpus import stopwords


In [ ]:
transform = T.Compose([
    T.Grayscale(num_output_channels=3),
    T.Resize((224, 224)),  
    T.ToTensor(),         
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  ])
data_location_train =  '../input/fer2013/train/'
data_location_test =  '../input/fer2013/test/'

train_dataset = datasets.ImageFolder(root= data_location_train, transform=transform)
test_dataset = datasets.ImageFolder(root=data_location_test, transform=transform)

In [ ]:
print("Classes:", train_dataset.classes) 
# def imshow(img):
#     img = img / 2 + 0.5      
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()

# dataiter = iter(train_data_loader)
# images, labels = next(dataiter)
# imshow(images[0])
# print(train_dataset.classes[labels[0]])

The following function will return train and test data loaders with a given batch size.

In [ ]:
def get_data_loaders(batch_size):
    train_data_loader = DataLoader(train_dataset, batch_size= batch_size, shuffle=False)
    test_data_loader = DataLoader(test_dataset, batch_size= batch_size, shuffle=False)
    return train_data_loader, test_data_loader

The following function will return an instance of the model that will be trained. Here `name` will be one of the following, `["alexnet", "vgg11", "resnet", "inception"]`.

In [ ]:
def get_model(model_name, num_classes):
    if model_name == "alexnet":
        model = models.alexnet(num_classes=num_classes, weights=None)

    elif model_name == "vgg11":
        model = models.vgg11(num_classes=num_classes, weights=None)

    elif model_name == "resnet18":
        model = models.resnet18(num_classes=num_classes, weights=None)

    elif model_name == "inceptionv3":
        model = models.inception_v3(num_classes=num_classes, weights=None)
        
    return model

The following functions will return the training criterion and optimizer respectively.

In [ ]:
def get_criterion():
    criterion = nn.CrossEntropyLoss()
    return criterion

def get_optimizer(model, learning_rate):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return optimizer

The following function will plot the training and test loss/accuracy vs epoch. You can use the `what` and `title` parameters to detect which learning curve is being plot. If you want, you can also save the plots for furture use.

The following function will train the model using the train loader and assess its performance on both the train and test data.

In [ ]:
def train(model, train_loader, test_loader, num_epochs, optimizer, criterion):
    model.to(device)
    train_accuracy_list = []
    train_loss_list = []
    test_accuracy_list = []
    test_loss_list = []

    for epoch in range(num_epochs):
        model.train()
        total = 0
        correct = 0
        train_loss = 0
        
        for img, labels in train_loader:
            img, labels = img.to(device), labels.to(device)
            optimizer.zero_grad()  # Add missing zero_grad() to reset gradients
            outputs = model(img)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
    
        train_acc = 100 * correct / total
        train_accuracy_list.append(train_acc)
        train_loss_list.append(train_loss / len(train_loader))

        model.eval()
        correct = 0
        total = 0
        test_loss = 0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        test_acc = 100 * correct / total
        test_accuracy_list.append(test_acc)
        test_loss_list.append(test_loss / len(test_loader))
                            
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss_list[-1]:.4f}, Test Loss: {test_loss_list[-1]:.4f}, Train Acc: {train_acc:.2f}%, Test Acc: {test_acc:.2f}%")
        
    return train_accuracy_list, train_loss_list, test_accuracy_list, test_loss_list


In [ ]:
def plot(train_values, test_values, what, title):
    plt.plot(train_values, label='Train')
    plt.plot(test_values, label='Test')
    plt.xlabel('Epochs')
    plt.ylabel(what)
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_sizes = [32, 64,128]
learning_rates = [0.01, 0.001,0.001]
names = ['alexnet', 'vgg11', 'resnet18', 'inceptionv3']
# modify the value in the following statement
num_epocs = 10
num_classes = 7
for bs in batch_sizes:
    for lr in learning_rates:
        for name in names:
            print(f"==========Training {name} with batch size {bs} and learning rate {lr}========")
            train_loader, test_loader = get_data_loaders(bs)
            model = get_model(name, num_classes)
            optimizer = get_optimizer(model,lr)
            criterion = get_criterion()

            train_acc, train_loss, test_acc, test_loss = train(
                model, train_loader, test_loader, num_epocs, optimizer, criterion)

            plot(train_acc, test_acc, what='acc', title=f'{name}_{bs}_{lr}_{"acc"}')
            plot(train_loss, test_loss, what='loss', title=f'{name}_{bs}_{lr}_{"loss"}')